In [15]:
# Personal Library
import project_baseball
# Pybaseball
import pybaseball as pb
# Packages
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
# Import Data 
mlb2020 = pd.read_csv('data/2020mlb.csv')
print('mlb 2020 data shape: {0}'.format(mlb2020.shape))

mlb 2020 data shape: (279660, 90)


# Feature Selection

---
The feature columns selected will be to help determine the what type of pitch is thrown. 

These features may also help in determining the type of pitch that will be thrown in sequence. I will also gather data on the players who are batting as much as possible.

In [6]:
# These columns are considered irrelevant as to having way too many nulls
mlb2020.drop(columns=['spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated','tfs_deprecated', 'tfs_zulu_deprecated',
       'umpire', 'sv_id','pitcher.1'], inplace=True)

In [7]:
mlb2020.head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,0,FF,2020-10-27,96.7,1.58,5.99,Julio Urias,642715.0,628711.0,strikeout,called_strike,4.0,Willy Adames called out on strikes.,W,R,L,LAD,TB,S,2.0,NaN,0.0,2.0,2020.0,0.2,1.6,-0.53,2.29,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,605131.0,-5.950264,-140.490456,-7.897391,3.772000,32.321911,-8.981441,3.50,1.69,NaN,NaN,NaN,95.4,2615.0,5.7,635886.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.82,NaN,NaN,0.0,1.0,0.0,0.0,NaN,65.0,3.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
1,1,FF,2020-10-27,94.1,2.91,5.45,Julio Urias,642715.0,628711.0,NaN,called_strike,1.0,NaN,W,R,L,LAD,TB,S,NaN,NaN,0.0,1.0,2020.0,0.8,1.3,-0.55,3.03,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,605131.0,-10.560246,-136.599519,-3.429867,11.723598,29.183810,-15.237217,3.50,1.69,NaN,NaN,NaN,93.4,2470.0,5.9,635886.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,2.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
2,2,FF,2020-10-27,94.9,1.77,6.02,Julio Urias,642715.0,628711.0,NaN,swinging_strike,2.0,NaN,W,R,L,LAD,TB,S,NaN,NaN,0.0,0.0,2020.0,0.2,1.5,-0.04,3.32,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,605131.0,-5.199252,-138.098234,-4.637970,4.158758,30.838499,-12.535677,3.50,1.69,NaN,NaN,NaN,94.0,2397.0,5.7,635886.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,1.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
3,3,FF,2020-10-27,94.4,1.66,5.93,Julio Urias,670712.0,628711.0,strikeout,called_strike,4.0,Mike Brosseau called out on strikes.,W,R,L,LAD,TB,S,2.0,NaN,3.0,2.0,2020.0,0.3,1.5,-0.37,2.15,NaN,NaN,NaN,1.0,9.0,Top,NaN,NaN,605131.0,-5.843595,-137.294295,-7.414897,4.754147,30.016237,-11.778755,3.34,1.53,NaN,NaN,NaN,93.7,2508.0,5.9,635886.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.60,NaN,NaN,0.0,1.0,0.0,0.0,NaN,64.0,6.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
4,4,CU,2020-10-27,81.4,1.46,6.06,Julio Urias,670712.0,628711.0,NaN,ball,13.0,NaN,W,R,L,LAD,TB,B,NaN,NaN,2.0,2.0,2020.0,-1.7,-0.2,-0.14,0.96,NaN,NaN,NaN,1.0,9.0,Top,NaN,NaN,605131.0,-0.383207,-118.447810,-4.454166,-15.553576,24.451936,-32.892744,3.34,1.53,NaN,NaN,NaN,80.2,3031.0,5.7,635886.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,5.0,Curveball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard


These will be the selected features.

```python
features = ['index', 'pitch_type','game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 
            'player_name', 'batter', 'events', 'description', 'stand', 'p_throws', 'balls', 'strikes',
            'type', 'bb_type', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az',
            'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'inning', 'inning_topbot',
            'effective_speed', 'release_spin_rate', 'pitcher','fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
            'fielder_7', 'fielder_8', 'fielder_9', 'at_bat_number', 'pitch_number', 'pitch_name',
            'bat_score', 'fld_score', 'post_bat_score', 'post_fld_score', 'if_fielding_alignment', 'of_fielding_alignment']
```

In [12]:
features = ['index', 'pitch_type','game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 
            'player_name', 'batter', 'events', 'description', 'stand', 'p_throws', 'balls', 'strikes',
            'type', 'bb_type', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az',
            'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'inning', 'inning_topbot',
            'effective_speed', 'release_spin_rate', 'pitcher','fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
            'fielder_7', 'fielder_8', 'fielder_9', 'at_bat_number', 'pitch_number', 'pitch_name',
            'bat_score', 'fld_score', 'post_bat_score', 'post_fld_score', 'if_fielding_alignment', 'of_fielding_alignment']

In [14]:
mlb2020[features].head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,events,description,stand,p_throws,balls,strikes,type,bb_type,vx0,vy0,vz0,ax,ay,az,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,inning,inning_topbot,effective_speed,release_spin_rate,pitcher,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,0,FF,2020-10-27,96.7,1.58,5.99,Julio Urias,642715.0,strikeout,called_strike,R,L,0.0,2.0,S,NaN,-5.950264,-140.490456,-7.897391,3.772000,32.321911,-8.981441,0.2,1.6,-0.53,2.29,NaN,NaN,NaN,9.0,Top,95.4,2615.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,65.0,3.0,4-Seam Fastball,1.0,3.0,1.0,3.0,Standard,Standard
1,1,FF,2020-10-27,94.1,2.91,5.45,Julio Urias,642715.0,NaN,called_strike,R,L,0.0,1.0,S,NaN,-10.560246,-136.599519,-3.429867,11.723598,29.183810,-15.237217,0.8,1.3,-0.55,3.03,NaN,NaN,NaN,9.0,Top,93.4,2470.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,65.0,2.0,4-Seam Fastball,1.0,3.0,1.0,3.0,Standard,Standard
2,2,FF,2020-10-27,94.9,1.77,6.02,Julio Urias,642715.0,NaN,swinging_strike,R,L,0.0,0.0,S,NaN,-5.199252,-138.098234,-4.637970,4.158758,30.838499,-12.535677,0.2,1.5,-0.04,3.32,NaN,NaN,NaN,9.0,Top,94.0,2397.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,65.0,1.0,4-Seam Fastball,1.0,3.0,1.0,3.0,Standard,Standard
3,3,FF,2020-10-27,94.4,1.66,5.93,Julio Urias,670712.0,strikeout,called_strike,R,L,3.0,2.0,S,NaN,-5.843595,-137.294295,-7.414897,4.754147,30.016237,-11.778755,0.3,1.5,-0.37,2.15,NaN,NaN,NaN,9.0,Top,93.7,2508.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,64.0,6.0,4-Seam Fastball,1.0,3.0,1.0,3.0,Standard,Standard
4,4,CU,2020-10-27,81.4,1.46,6.06,Julio Urias,670712.0,NaN,ball,R,L,2.0,2.0,B,NaN,-0.383207,-118.447810,-4.454166,-15.553576,24.451936,-32.892744,-1.7,-0.2,-0.14,0.96,NaN,NaN,NaN,9.0,Top,80.2,3031.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,64.0,5.0,Curveball,1.0,3.0,1.0,3.0,Standard,Standard
